# Principal null directions in Kerr spacetime

This notebook demonstrates a few capabilities of SageMath in computations regarding Kerr spacetime. More precisely, it focuses on the principal null directions. This notebook makes use of SageMath tools developed through the [SageManifolds](https://sagemanifolds.obspm.fr) project.

*NB:* a version of SageMath at least equal to 8.2 is required to run this notebook:

In [ ]:
version()

First we set up the notebook to display mathematical objects using LaTeX rendering:

In [ ]:
%display latex

To speed up the computations, we ask for running them in parallel on 8 cores:

In [ ]:
Parallelism().set(nproc=8)

## Spacetime manifold

We declare the Kerr spacetime (or more precisely the Boyer-Lindquist domain of Kerr spacetime) as a 4-dimensional Lorentzian manifold:

In [ ]:
M = Manifold(4, 'M', latex_name=r'\mathcal{M}', structure='Lorentzian')
print(M)

Let us declare the **Boyer-Lindquist coordinates** via the method `chart()`, the argument of which is a string expressing the coordinates names, their ranges (the default is $(-\infty,+\infty)$) and their LaTeX symbols:

In [ ]:
BL.<t,r,th,ph> = M.chart(r't r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi') 
print(BL) ; BL

In [ ]:
BL[0], BL[1]

<h2>Metric tensor</h2>

<p>The 2 parameters $m$ and $a$ of the Kerr spacetime are declared as symbolic variables:</p>

In [ ]:
var('m, a', domain='real')

We get the (yet undefined) spacetime metric by 

In [ ]:
g = M.metric()

<p>The metric is set by its components in the coordinate frame associated with Boyer-Lindquist coordinates, which is the current manifold's default frame:</p>

In [ ]:
rho2 = r^2 + (a*cos(th))^2
Delta = r^2 -2*m*r + a^2
g[0,0] = -(1-2*m*r/rho2)
g[0,3] = -2*a*m*r*sin(th)^2/rho2
g[1,1], g[2,2] = rho2/Delta, rho2
g[3,3] = (r^2+a^2+2*m*r*(a*sin(th))^2/rho2)*sin(th)^2
g.display()

<p>A matrix view of the components with respect to the manifold's default vector frame:</p>

In [ ]:
g[:]

<p>The list of the non-vanishing components:</p>

In [ ]:
g.display_comp()

<h2>Levi-Civita Connection</h2>

<p>The Levi-Civita connection $\nabla$ associated with $g$:</p>

In [ ]:
nabla = g.connection() ; print(nabla)

<p>Let us verify that the covariant derivative of $g$ with respect to $\nabla$ vanishes identically:</p>

In [ ]:
nabla(g).display()

<h2>Killing vectors</h2>
<p>The default vector frame on the spacetime manifold is the coordinate basis associated with Boyer-Lindquist coordinates:</p>

In [ ]:
M.default_frame() is BL.frame()

In [ ]:
BL.frame()

<p>Let us consider the first vector field of this frame:</p>

In [ ]:
xi = BL.frame()[0] ; xi

In [ ]:
print(xi)

<p>The 1-form associated to it by metric duality is</p>

In [ ]:
xi_form = xi.down(g) ; xi_form.display()

<p>Its covariant derivative is</p>

In [ ]:
nab_xi = nabla(xi_form) ; print(nab_xi) ; nab_xi.display()

<p>Let us check that the Killing equation is satisfied:</p>

In [ ]:
nab_xi.symmetrize() == 0

<p>Similarly, let us check that $\frac{\partial}{\partial\phi}$ is a Killing vector:</p>

In [ ]:
chi = BL.frame()[3] ; chi

In [ ]:
nabla(chi.down(g)).symmetrize() == 0

## Principal null vectors

Let us consider the following vector fields $k$ and $\ell$, defined from their components with respect to Boyer-Lindquist coordinates:

In [ ]:
k = M.vector_field(name='k')
k[:] = [(r^2+a^2)/(2*rho2), -Delta/(2*rho2), 0, a/(2*rho2)]
k.display()

In [ ]:
el = M.vector_field(name='el', latex_name=r'\ell')
el[:] = [(r^2+a^2)/Delta, 1, 0, a/Delta]
el.display()

Let us check that $k$ and $\ell$ are null vectors:

In [ ]:
g(k,k).expr()

In [ ]:
g(el,el).expr()

Their scalar product is $-1$:

In [ ]:
g(k,el).expr()

Note that the scalar product (with respect to metric $g$) can also be computed by means of the method `dot`:

In [ ]:
k.dot(el).expr()

Let us evaluate the "acceleration" of $k$, i.e. $\nabla_k k$:

In [ ]:
acc_k = nabla(k).contract(k)
acc_k.display()

We check that $k$ is a pregeodesic vector, i.e. that $\nabla_k k = \kappa_k k$ for some scalar field $\kappa_k$:

In [ ]:
for i in [0,1,3]:
    show(acc_k[i] / k[i])

In [ ]:
kappa_k = acc_k[[0]] / k[[0]]
kappa_k.display()

In [ ]:
acc_k == kappa_k * k

Similarly let us evaluate the "acceleration" of $\ell$:

In [ ]:
acc_l = nabla(el).contract(el)
acc_l.display()

Hence $\ell$ obeys $\nabla_\ell\, \ell=0$, i.e. it is a geodesic vector.

### Check that $k$ and $\ell$ are doubly degenerate principal null vectors

$k$ defines a **repeated null direction** (one says that $k$ is a **doubly degenerate principal null vector**) iff it obeys the following identity:
$$ C^a_{\ \, mn[b} k_{c]} k^m k^n = 0 \qquad \mbox{(1)},$$
where $C$ is the Weyl tensor of the metric $g$.

The Weyl tensor is computed via

In [ ]:
C = g.weyl()
print(C)

For instance, the component $C^0_{\ \, 010} = C^t_{\ \, trt}$  is

In [ ]:
C[0, 1, 0, 1]

We can check the identity (1) in a single line as follows:

In [ ]:
(C.contract(1, 2, k*k, 0, 1) * k.down(g)).antisymmetrize(1, 2).display()

Alternatively, we may check (1) by using index notations.
First, we introduce the tensor $A^a_{\ \, b} = C^a_{\ \, mnb} k^m k^n$:

In [ ]:
A = C['^a_{mnb}'] * (k*k)['^{mn}']
print(A)

and the 1-form $k_a = g_{am} k^m$:

In [ ]:
kf = g['_{am}'] * k['^m']
print(kf)

Then (1) is equivalent to $A^a_{\ \, [b} k_{c]} = 0$, which is readily checked as

In [ ]:
(A*kf)['^a_{[bc]}'].display()

Similarly, let us check that $\ell$ obeys (1) as well:

In [ ]:
(C.contract(1, 2, el*el, 0, 1) * el.down(g)).antisymmetrize(1, 2).display()